# Parkinson's Disease Progression Prediction with TensorFlow Decision Forests

  

# Import the Required Libraries

In [1]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math
import xgboost as xg
import pickle
from sklearn.preprocessing import OrdinalEncoder,StandardScaler,MinMaxScaler
import os

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Load the Dataset

First, let us load the dataset using Pandas.

In [2]:
from numpy.random import seed
seed(1123) 
import random as pyrandom
pyrandom.seed(1123)

In [3]:
# Load a dataset into a Pandas DataFrame
import os
os.chdir(r'/kaggle/input/amp-parkinsons-disease-progression-prediction')
train_proteins = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv")
train_peptides = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv")
train_clinical = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv")


# Prepare the dataset

We have to predict the ratings for the the first 4 segments of UPDRS `updrs_1`, `updrs_2`, `updrs_3` and `updrs_4` that is likely to be recorded by the clinic during a patient visit. Hence these are our labels.

In [4]:
# Function to prepare dataset with all the steps mentioned above:
def prepare_dataset(train_proteins, train_peptides):
    # Step 1: Grouping 
    df_protein_grouped = train_proteins.groupby(['visit_id','UniProt'])['NPX'].mean().reset_index()
    df_peptide_grouped = train_peptides.groupby(['visit_id','Peptide'])['PeptideAbundance'].mean().reset_index()
    
    # Step 2: Pivoting
    df_protein = df_protein_grouped.pivot(index='visit_id',columns = 'UniProt', values = 'NPX').rename_axis(columns=None).reset_index()
    df_peptide = df_peptide_grouped.pivot(index='visit_id',columns = 'Peptide', values = 'PeptideAbundance').rename_axis(columns=None).reset_index()
    
    # Step 3: Merging
    pro_pep_df = df_protein.merge(df_peptide, on = ['visit_id'], how = 'left')
    
    return pro_pep_df   

In [5]:
pro_pep_df = prepare_dataset(train_proteins, train_peptides)

dataset_df = pro_pep_df.merge(train_clinical[['visit_id', 'patient_id', 'visit_month', "updrs_1", "updrs_2", "updrs_3", "updrs_4"]], 
                              on = ['visit_id'], how = 'left')

d1=dataset_df.iloc[:,-6:].copy(deep=True)
d1['visit_id']=dataset_df['visit_id']

d2=d1.pivot(columns='visit_month',values=["updrs_1", "updrs_2", "updrs_3", "updrs_4"]).rename_axis() #.reset_index()

# print("pivoted")
# print("pivoted with visit months columns \n",d2.info())

d3=d2.to_numpy()

c=[0,16,32,48]

# d3=d3[:,~c ]
d3=np.delete(d3,c,axis=1)



label_values=pd.DataFrame(d3)

# label_values=labl.dropna(axis=1,how='all')

d5=pd.concat([dataset_df,label_values],axis=1)



# print("df after concat \n",d5.info())

d6=d5.drop(['visit_id','patient_id'],axis=1)

# print(d6)
# d6.drop(["updrs_1", "updrs_2", "updrs_3", "updrs_4"],axis=1,inplace=True)



# # d10.iloc[:,-59:].fillna(mea,inplace=True)
d6.iloc[:,-59:].fillna(d6.iloc[:,-59:].mean(),inplace=True)

d6.fillna(d6.mean(),inplace=True)

# # print("final df \n",d6.info())
# # print("final df \n",d6)
mn=d6.iloc[:,:-65].mean() 

In [6]:
ind_dict={}
i=0
for u in ["updrs_1", "updrs_2", "updrs_3", "updrs_4"]:
    for m in [0,3,6,12,18,24,30,36,48,54,60,72,84,96,108]:
#         print(u,m)
        ind_dict[(u,m)]=i
        i+=1
# ind_dict
        

# Basic exploration of the prepared dataset

Our feature engineered dataset is composed of 1196 columns and 1113 entries.

Let us print out the first 5 entries using the following code:

Now let us write a function to split the dataset into train and test datasets. This will be used in the training loop.

In [7]:
# def split_dataset(dataset, test_ratio=0.05):
#   test_indices = np.random.rand(len(dataset)) < test_ratio
#   return dataset[~test_indices], dataset[test_indices]

Before training the model, let us write a function to calculate the scoring metric of the competition.

# sMAPE

The scoring metric for this competition is sMAPE(symmetric Mean Absolute Percentage Error). In MAPE(Mean Absolute Percentage Error) we calculate the sum of absolute value of the forecast errors, divided by the actual values for each period([source](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)). 

Difference between MAPE and sMAPE is that sMAPE has both a lower bound and an upper bound. 


You can read more about sMAPE [here](https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error).

Since TensorFlow Datasets doesn't have a bultin function to calculate sMAPE metric yet, we will write a custom function to calculate sMAPE.

In [8]:
def smape(A, F):
    A=np.array(A)+1
    F=np.array(F)+1
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))


# def smape(predt, y):
#     
#     y[y<=0] = 1e-6
#     grad = 200*np.sign(predt-y)*(np.sign(predt)+1)*y/np.power(y+np.abs(predt),2)
#     hess = -400*np.sign(predt-y)*(np.sign(predt)+1)*y/np.power(y+np.abs(predt),3)
#     return grad, hess

# def smape1(predt, y):     
 
#     predt=predt+1
#     y=y+1
#     grad = 200*np.sign(predt-y)*(np.sign(predt)+1)*y/np.power(y+np.abs(predt),2)
#     hess = -400*np.sign(predt-y)*(np.sign(predt)+1)*y/np.power(y+np.abs(predt),3)
#     return grad, hess


# def Metric_SMAPE_xgb(y_pred,true):
# y_true = true.get_label()
# denominator = (y_true + np.abs(y_pred)) / 200.0
# diff = np.abs(y_true - y_pred) / denominator
# return 'pred',np.mean(diff)

 

# # Define the SMAPE loss function
# def smape_obj(y_true, y_pred):
#     numerator = np.abs(y_true - y_pred)
#     denominator = np.abs(y_true) + np.abs(y_pred)
#     grad = np.where(denominator > 0, 2.0 * numerator / denominator, 0.0)
#     hess = np.where(denominator > 0, 2.0 * numerator / np.square(denominator), 0.0)
#     return grad, hess

# import numpy as np

# Define the SMAPE loss function
# def smape_obj(y_true, y_pred):
#     epsilon = 0.1
#     numerator = np.abs(y_true - y_pred)
#     denominator = np.abs(y_true) + np.abs(y_pred) + epsilon
#     grad = np.where(y_true > y_pred, -2.0 * numerator / denominator, 2.0 * numerator / denominator)
#     hess = 2.0 * epsilon / np.square(denominator)
#     return grad, hess

 

# # Define the SMAPE loss function
# def smape_obj(y_true, y_pred):
#     epsilon = 0.1
#     numerator = np.abs(y_true - y_pred)
#     denominator = np.abs(y_true) + np.abs(y_pred) + epsilon
#     sign = np.sign(y_true - y_pred)
#     grad = -sign * (2.0 * numerator / denominator ** 2)
#     hess = 2.0 * epsilon / denominator ** 3
#     return grad, hess



# import numpy as np
# from xgboost.core import XGBoostError

# # Define the SMAPE loss function
# def smape_obj(y_true, y_pred):
#     epsilon = 0.1
#     numerator = np.abs(y_true - y_pred)
#     denominator = np.abs(y_true) + np.abs(y_pred) + epsilon
#     sign = np.sign(y_true - y_pred)
#     grad = -sign * (2.0 * numerator / denominator ** 2)
#     hess = 2.0 * epsilon / denominator ** 3
#     return grad, hess

# # Define a wrapper function for the SMAPE loss that can be used with XGBoost
# def xgboost_smape(y_true, y_pred):
# #     y_true = y_true.get_label()
#     grad, hess = smape_obj(y_true, y_pred)
#     return grad, hess

# # # Example usage with XGBoost
# # import xgboost as xgb
# # from sklearn.datasets import load_boston

# # data = load_boston()
# # X = data['data']
# # y = data['target']

# # dtrain = xgb.DMatrix(X, y)
# # params = {'objective': xgboost_smape, 'eval_metric': 'mae'}
# # bst = xgb.train(params, dtrain)



# Training

We will train a model each to predict the first 4 segments of the UPDRS, `updrs_1`, `updrs_2`, `updrs_3` and `updrs_4`. Thus we we will be training 4 models, one for each label.

We need to provide a few data structures to our training loop to store the trained models, sMAPE scores and mse scores for the trained models.

We will create these using the following code:

In [9]:
# Create an empty dictionary to store the models trained for each label.
model_dict = {}

# Create an empty dictionary to store the mse score of the models trained for each label.
mse_dict = {}

# Create an empty dictionary to store the sMAPE scores of the models trained for each label.
smape_dict = {}

Next, we will create a feature list with names of the columns we will use for training the model. We will avoid the column `visit_id` and add column the `visit_month`

In [10]:
FEATURES = [i for i in pro_pep_df.columns if i not in ["visit_id"]]
FEATURES.append("visit_month")
# FEATURES[-10:]

Now let us train the models.

In [11]:
# xgr=xg.XGBRegressor(objective = 'reg:absoluteerror',eval_metric='mape',colsample_bytree=0.4603, gamma=0.0468, 
#                              learning_rate=0.05, max_depth=3, 
#                              min_child_weight=1.7817, n_estimators=2200,
#                              reg_alpha=0.4640, reg_lambda=0.8571,
#                              subsample=0.5213, silent=1,
#                              random_state =1123, nthread = -1,verbosity=1,tree_method="gpu_hist")

xgr=xg.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=4, 
                             min_child_weight=1.7817, n_estimators=1000,
                    reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =1123, nthread = -1,verbosity=1) #,tree_method="gpu_hist")
# number of estimators, max_depth, min child weight, gamma


# xgr=xg.XGBRegressor(objective='reg:absoluteerror',  n_estimators=2200, 
#                              random_state =1123, nthread = -1,verbosity=1) #,tree_method="gpu_hist")



#  lgb_params = {  'objective' : 'regression',
#                    'learning_rate' : 0.005,
#                    'max_depth' : -1,
#                    'num_leaves' : 5,
#                    'min_child_samples':25,
#                    'reg_alpha' : 25,
#                    'reg_lambda' : 25,
#                    'n_estimators' : 2500,
#                    'colsample_bytree' : 1,
#                    'subsample' : 1,
#                      'random_state' : 2021}

# xgr=xg.XGBRegressor(objective = 'reg:absoluteerror',colsample_bytree=1, gamma=0.0468, #num_leaves=5,
#                              learning_rate=0.005, max_depth=0, #min_child_samples=25,min_child_weight=1.7817,
#                               n_estimators=2500,
#                     reg_alpha=25, reg_lambda=25, 
#                              subsample=1, silent=1,
#                              random_state =1123, nthread = -1,verbosity=1,tree_method="gpu_hist")


# XGBRegressor(base_score=None, booster=None, callbacks=None,
#              colsample_bylevel=None, colsample_bynode=None,
#              colsample_bytree=None, early_stopping_rounds=None,
#              enable_categorical=False, eval_metric=None, feature_types=None,
#              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
#              interaction_constraints=None, learning_rate=None, max_bin=None,
#              max_cat_threshold=None, max_cat_to_onehot=None,
#              max_delta_step=None, max_depth=None, max_leaves=None,
#              min_child_weight=None, missing=nan, monotone_constraints=None,
#              n_estimators=2200, n_jobs=None, nthread=-1, num_parallel_tree=None,
#              objective='reg:absoluteerror', ...)


# xgr=xg.XGBRegressor(colsample_bytree=1, gamma=0.0468, #num_leaves=5,
#                              learning_rate=0.005, max_depth=20, #min_child_samples=25,min_child_weight=1.7817,
#                               n_estimators=2500,
#                     reg_alpha=25, reg_lambda=25, 
#                              subsample=1, silent=1,
#                              random_state =1123, nthread = -1,verbosity=1) #,tree_method="gpu_hist")

In [12]:
# from sklearn.ensemble import RandomForestRegressor    #,  GradientBoostingRegressor
# from sklearn.ensemble import HistGradientBoostingRegressor
# rf=RandomForestRegressor()
# hg=HistGradientBoostingRegressor()

In [13]:
import pickle
os.chdir(r'/kaggle/working/')
model_dict1={}
# List of target labels to loop through and train models
target = ["updrs_1", "updrs_2", "updrs_3", "updrs_4"]    
for u in target:
    df=d6.copy(deep=True)   
    
    
    
    y_train = df.loc[:,u]

    X_train = df.drop(columns=list(df.iloc[:,-64:].columns.values),axis=0) 
    
# #     scaler = StandardScaler()
#     scaler=MinMaxScaler()
#     sc = scaler.fit(X_train)
#     X_train = sc.transform(X_train)
    
     
    xgr.fit(X_train,y_train) 

#         Store the model
    filename= str(u)
    pickle.dump(xgr,open(filename, 'wb')) 
#         xgr.save_model(filename)  

    model_dict1[u]=filename
    print("model for ",u,"saved in file name ",filename) 

[05:17:20] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

model for  updrs_1 saved in file name  updrs_1
[05:17:57] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

model for  updrs_2 saved in file name  updrs_2
[05:18:35] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

model for  updrs_3 saved in file name  updrs_3
[05:19:12] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

model for  updrs_4 saved in file name  updrs_4


In [14]:
# import pickle
model_dict={}
# List of target labels to loop through and train models
target = ["updrs_1", "updrs_2", "updrs_3", "updrs_4"]

# Loop through each label   #[0,3,6,12,18,24,30,36,48,54,60,72,84,96,108]
# #0 -> 0,6,12,24
# #6 -> 6,12,18,30
# #12 > 12,18,30,36
# #24 -> 24,30,36,48
empty=[]

for m in [0,6,12,18,24,30,36,48,60,72,84,96,108]:
#     df=d6.copy(deep=True)
    
    for u in target:
        df=d6.copy(deep=True)

        # Merge the label 'visit_id', 'patient_id', 'visit_month' and label columns from `train_clinical` 
        # data frame to `pro_prep_df` data frame on the `visit_id` column.
#         dataset_df = pro_pep_df.merge(train_clinical[['visit_id', 'patient_id', 'visit_month', label]], on = ['visit_id'], how = 'left')

#         # Drop null value label rows
#         dataset_df = dataset_df.dropna(subset=[label])

#         # Make a new copy of the FEATURES list we created previously. Add `label` to it.
#         feature_list = FEATURES.copy() 
#         feature_list.append(label)
#         print("before choose ",df.shape)
        df = df[df['visit_month']==m]
#         print("df after vm ",df.shape)
    
        df.dropna(inplace=True, axis=0, subset=[ind_dict[u,m]])
#         print("after dropna ",df.shape)
    
        print("label index \n",u,m,ind_dict[u,m])
            
       
        # Split the dataset into train and validation datasets.
#         train_df, valid_df = split_dataset(df)
#         print("{} examples in training, {} examples in testing.".format(len(train_df), len(valid_df)))
        
        
#         if ((len(train_df)==0) or (len(valid_df)==0)): 
        if (len(df)==0): 
            empty.append((u,m))
#             print("{} examples in training, {} examples in testing.".format(len(train_df), len(valid_df)),"skipping ")
            print("{} examples in training, {} examples in testing.".format(len(df),"skipping "))
            break

    #     rf = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION, verbose=0)
    #     rf.compile(metrics=["mse"])

        # Train the model.
    #     rf.fit(x=train_ds)
    
#         d6.loc[:,ind_dict[u,m]]

        
    
        
#         y_train = train_df.loc[:,ind_dict[u,m]]
        y_train = df.loc[:,ind_dict[u,m]]
        
#         print("ytrain ",y_train)
        print("check sum \n",np.sum(df.loc[:,ind_dict[u,m]]==np.nan))
    
#         print("ytrain \n",y_train)
    
#     d7.drop(columns=list(d6.iloc[:,-59:].columns.values),inplace=True,axis=0)

#         X_train = train_df.drop( columns=list(train_df.iloc[:,-59:].columns.values),axis=0)
        X_train = df.drop( columns=list(df.iloc[:,-64:].columns.values),axis=0)
    
# #         scaler = StandardScaler()
#         scaler=MinMaxScaler()
#         sc = scaler.fit(X_train)
#         X_train = sc.transform(X_train)
    
#         print("xtrain ",X_train)
    
        
    
#         print("X_train \n",X_train)

#         y_val = valid_df.loc[:,ind_dict[u,m]]
#         X_val = valid_df.drop(columns=list(train_df.iloc[:,-59:].columns.values),axis=0)

#         model_dict[(u,m)] = xgr
        xgr.fit(X_train,y_train)
    
        

    #     rf.fit(X_train,y_train)
    #     hg.fit(X_train,y_train)

#         Store the model
        filename= str(u)+"_"+str(m) 
        pickle.dump(xgr,open(filename, 'wb')) 
#         xgr.save_model(filename)  
        
        model_dict[(u,m)]=filename
        print("model for ",u,m,"saved in file name ",filename)
        
#         model_dict[(u,m)] = xgr
#         
#         print("ytrain predict for ",u,m,"\n","df.shape ",df.shape, "xtrain shape ",X_train.shape, "\n",xgr.predict(X_train),"\n")
#         print("ytrain \n",y_train)
    #     model_dict[label] = rf
    #     model_dict[label] = hg 
    #     mse_dict[label] = evaluation["mse"]   
#         preds = xgr.predict(X_val)
    #     preds=rf.predict(X_val)
    #     preds=hg.predict(X_val)
#         smape_dict[(u,m)] = smape(y_val.values.tolist(), preds.flatten())


        
        
        
        
print("empty learners ",empty)

label index 
 updrs_1 0 0
check sum 
 0
[05:19:50] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

model for  updrs_1 0 saved in file name  updrs_1_0
label index 
 updrs_2 0 15
check sum 
 0
[05:20:02] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

model for  updrs_2 0 saved in file name  updrs_2_0
label index 
 updrs_3 0 30
check sum 
 0
[05:20:15] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

model for  updrs_3 0 saved in file name  updrs_3_0
label index 
 updrs_4 0 45
check sum 
 0
[05:20:28] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

model for  updrs_4 0 saved in file name  updrs_4_0
label index 
 updrs_1 6 2
check sum 
 0
[05:20:41] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

model for  updrs_1 6 saved in file name  updrs_1_6
label index 
 updrs_2 6 17
check sum 
 0
[05:20:50] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used

In [15]:
!zip -r "park-xgb-final-1305.zip" /kaggle/working/
from IPython.display import FileLink
FileLink(r'park-xgb-final-1305.zip')

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/updrs_3_0 (deflated 80%)
  adding: kaggle/working/updrs_4_72 (deflated 91%)
  adding: kaggle/working/updrs_1_108 (deflated 96%)
  adding: kaggle/working/updrs_3_24 (deflated 80%)
  adding: kaggle/working/updrs_2_30 (deflated 97%)
  adding: kaggle/working/updrs_1_84 (deflated 90%)
  adding: kaggle/working/updrs_1_72 (deflated 92%)
  adding: kaggle/working/updrs_3_72 (deflated 91%)
  adding: kaggle/working/updrs_1_96 (deflated 97%)
  adding: kaggle/working/updrs_1_18 (deflated 94%)
  adding: kaggle/working/updrs_2_12 (deflated 81%)
  adding: kaggle/working/updrs_2_36 (deflated 81%)
  adding: kaggle/working/updrs_3_108 (deflated 96%)
  adding: kaggle/working/updrs_1_60 (deflated 84%)
  adding: kaggle/working/updrs_1_24 (deflated 81%)
  adding: kaggle/working/updrs_2_0 (deflated 80%)
  adding: kaggle/working/updrs_4_36 (deflated 82%)
  adding: kaggle/working/updrs_4_18 (deflated 95%)
  adding: kaggle/working/updrs_2_96 (deflate

/kaggle/working/park-xgb-final-1305.zip

# Inspect the sMAPE scores of the models

In [16]:
# for name, value in smape_dict.items():
#   print(f"label {name}: sMAPE {value:.4f}")

# print("\nAverage sMAPE", sum(smape_dict.values())/len(16)

# label updrs_1: sMAPE 61.4490
# label updrs_2: sMAPE 95.1665
# label updrs_3: sMAPE 83.4445
# label updrs_4: sMAPE 154.8876

# Average sMAPE 98.73688203627106

# label updrs_1: sMAPE 66.3694
# label updrs_2: sMAPE 82.9369
# label updrs_3: sMAPE 76.4665
# label updrs_4: sMAPE 140.2210

# Average sMAPE 91.49843641219168

# Submission

In [17]:
# import pickle
# os.chdir(r'/kaggle/input/draftmod')
# model_dict1={}
# # List of target labels to loop through and train models
# target = ["updrs_1", "updrs_2", "updrs_3", "updrs_4"]    
# for u in target:
     
#     filename= str(u)
#     model_dict1[u]=pickle.load(open(filename, 'rb')) 
    
# #     pickle.load(open(model_dict1[u],'rb'))
# #         xgr.save_model(filename)  
# model_dict1

In [18]:
# # import pickle
# model_dict={}
# # List of target labels to loop through and train models
# target = ["updrs_1", "updrs_2", "updrs_3", "updrs_4"]

# for m in [0,6,12,18,24,30,36,48,60,72,84,96,108]:
 
    
#     for u in target:
#         filename= str(u)+"_"+str(m) 
#         model_dict[(u,m)]=pickle.load(open(filename, 'rb')) 
# model_dict

In [19]:
#     df=d6.copy(deep=True)

#     # Merge the label 'visit_id', 'patient_id', 'visit_month' and label columns from `train_clinical` 
#     # data frame to `pro_prep_df` data frame on the `visit_id` column.
# #         dataset_df = pro_pep_df.merge(train_clinical[['visit_id', 'patient_id', 'visit_month', label]], on = ['visit_id'], how = 'left')

# #         # Drop null value label rows
# #         dataset_df = dataset_df.dropna(subset=[label])

# #         # Make a new copy of the FEATURES list we created previously. Add `label` to it.
# #         feature_list = FEATURES.copy() 
# #         feature_list.append(label)
# #         print("before choose ",df.shape)
#     u='updrs_1'
#     m=6
#     df = df[df['visit_month']==m]
# #         print("df after vm ",df.shape)

#     df.dropna(inplace=True, axis=0, subset=[ind_dict[u,m]])
# #         print("after dropna ",df.shape)

#     print("label index \n",u,m,ind_dict[u,m])


#     # Split the dataset into train and validation datasets.
# #         train_df, valid_df = split_dataset(df)
# #         print("{} examples in training, {} examples in testing.".format(len(train_df), len(valid_df)))


# # #         if ((len(train_df)==0) or (len(valid_df)==0)): 
# #     if (len(df)==0): 
# #         empty.append((u,m))
# # #             print("{} examples in training, {} examples in testing.".format(len(train_df), len(valid_df)),"skipping ")
# #         print("{} examples in training, {} examples in testing.".format(len(df),"skipping "))
# #         break

# #     rf = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION, verbose=0)
# #     rf.compile(metrics=["mse"])

#     # Train the model.
# #     rf.fit(x=train_ds)

# #         d6.loc[:,ind_dict[u,m]]




# #         y_train = train_df.loc[:,ind_dict[u,m]]

    
#     y_train = df.loc[:,ind_dict[u,m]]

# #         print("ytrain ",y_train)
#     print("check sum \n",np.sum(df.loc[:,ind_dict[u,m]]==np.nan))

# #         print("ytrain \n",y_train)

# #     d7.drop(columns=list(d6.iloc[:,-59:].columns.values),inplace=True,axis=0)

# #         X_train = train_df.drop( columns=list(train_df.iloc[:,-59:].columns.values),axis=0)
#     X_train = df.drop( columns=list(df.iloc[:,-64:].columns.values),axis=0)

# #     scaler = StandardScaler()
# #     sc = scaler.fit(X_train)
# #     X_train = sc.transform(X_train)

# #       


In [20]:
# master_df=pd.DataFrame()

# FEATURES = [i for i in pro_pep_df.columns if i not in ["visit_id"]]
# FEATURES.append("visit_month")

def get_predictions(test_df, master_testdf, model_dict):
#     os.chdir(r'/kaggle/input/draftmod')
    
#     master_df=pd.concat([master_df,test_df])
    
#     test_df=np.where(test_df[FEATURES].sum(axis=1) == 0,np.where(test_df))

    # Target labels
    target = ["updrs_1", "updrs_2", "updrs_3", "updrs_4"]
    
#     t=test_df.copy(deep=True)
    
#     test_df=np.where(test_df.isna(),mn,test_df)

    temp = test_df[FEATURES].copy()
    mask = test_df[FEATURES].sum(axis=1) != 0
    new_testdf = temp.loc[mask]
    
    master_testdf=master_testdf.sort_values(by=['patient_id','visit_month'],ascending=False)
    
#     print(" values fasdfasd 1 ",test_df[FEATURES].sum(axis=1))
#     print(" values fasdfasd 2 ",test_df[FEATURES].sum(axis=1).any())
#     print(" values fasdfasd 4 ",test_df[FEATURES].sum(axis=1).any(axis=0))
#     print(" values fasdfasd 4",test_df[FEATURES].sum(axis=1).any(axis=1))
#     dfasd
    
    
    if test_df[FEATURES].sum(axis=1).any():
        print("in loop check for 0")
#         if test_df.loc[0,'visit_month']==0
#             mask = test_df[FEATURES].sum(axis=1) != 0 
#             new_testdf = temp.loc[mask]
            
#         else:
        mask=test_df[FEATURES].iloc[:,:].sum(axis=1) == 0
    
        print("mask is ",mask)
        temp_nan=test_df.loc[mask]
        print("temp nan ",temp_nan)
        if temp_nan.empty==False:
            print("temp nan ",temp_nan)
        
            patientids = list(temp_nan.patient_id)
            print(patientids)
#             sdf
            mvm=max(master_testdf.visit_month[master_testdf.patient_id.isin(patientids)])
            print("max vm is ",mvm)
            print("master df is \n",master_testdf)
            print("master_testdf.patient_id ",master_testdf.patient_id)
            print("master_testdf.patient_id.isin(patientids) ",master_testdf.patient_id.isin(patientids))
            
            
            
            
            m1=master_testdf.patient_id.isin(patientids)
            a1 =master_testdf.loc[m1]
            m2=master_testdf.visit_month==mvm
            b1=master_testdf.loc[m2]
            print("check 1 ",a1)
            print("check 2 ",b1)
            
            common_values = set(a1['visit_id']) & set(b1['visit_id'])
            rep_df = b1[b1['visit_id'].isin(common_values)] 
            print('replace \n',rep_df) 
            
            
            print(test_df)
             
            new_testdf=test_df[FEATURES].append(rep_df[FEATURES])
            
            test_df=test_df.append(rep_df)
            
            print("new tdf \n",new_testdf)
            
#             mask = test_df[FEATURES].sum(axis=1) != 0 
#             new_testdf=newdf[FEATURES].loc[mask]
            print("nan replaced ehre \n",new_testdf, "type",type(new_testdf))
    
#             adffad

#     else: new_testdf=temp.loc[mask]
        
    
    
#     temp=test_df[FEATURES].copy()
#     new_test_df=test_df[FEATURES].copy()
    
#     print("columns of newtestdf ",new_test_df.columns.values)
    
     
#     new_test_df.drop(columns=['visit_month'],inplace=True) #,axis=0)
    
    
#     new_test_df=pd.DataFrame(np.where(new_test_df.isna(),mn,new_test_df))
#     visitmonth=pd.DataFrame(t['visit_month'])
    
#     new_test_df = pd.concat([new_test_df,visitmonth],axis=0)
    
     
#     print("temp before \n",temp)
    print("test df \n",test_df)
    print("new test df in after mask \n",new_testdf, type(new_testdf)) 
#     adfda
#     print("test df \n",test_df)
    
#     dsadf
#     newdf=new_test_df.copy(deep=True)
#     testdf=test_df.copy(deep=True)
    
#     months = 
     
    
    l=test_df.shape[0] 
    
#     print("vm values ",a)
#     afds
    vm=test_df.loc[0,'visit_month'] 
    
    print("type of vm here :",type(vm))
    
    
    if isinstance(vm,pd.Series):
        vm=vm.max()
        
    else:
        vm=vm
#         vm=max(list(test_df.loc[0,'visit_month'].tolist()))
    
#     print("vm value ",vm)
#     asdfasd

#     import random
#     vm1=test_df.loc[0,'visit_month'] 
#     vm2=test_df.loc[random.randrange(l+1),'visit_month'] 
    
#     if vm1==vm2: vm=vm1
#     mask = test_df[FEATURES].sum(axis=1) != 0
    result = pd.DataFrame()    
    for pm in [0,6,12,24]: 
        
#         vm=0
        m=pm+vm
        
        
        print("predicting for visiting month ",vm,"/n","prediction month hence \n",pm, "dictionary month",m) 

        for u in target:
            
            test_data=new_testdf.copy(deep=True)
                
            test_df['result_' + str(u)] = 0
            
#             if u != 'updrs_4':
#                 print(model_dict[u])
    #             test_df.loc[mask, 'result_' + str(u)] = model_dict[u].predict(test_ds)
#                 print("new test df in u loop \n",new_test_df.head())
        
            temp = test_df[["visit_id", "result_" + str(u)]].copy()  #visit_id
            

            temp["prediction_id"] = temp["visit_id"] + "_"+str(u) + "_plus_" + str(pm) + "_months"
            
            temp["rating"]=0
            
            
#             print("temp \n",temp)
#             asdfa
#             temp=temp.loc[mask]
#             print("temp afer maks \n",temp)
#                 print("temp here in u loop \n",temp)
            print("test_data used for prediction   \n",test_data, "shape ",test_data.shape, "type",type(test_data))
            print("mean is ",mn)
    
            test_data.fillna(mn,inplace=True)
        
            print("after mean ",test_data)
            
            
            
# #             scaler=StandardScaler()
#             scaler=MinMaxScaler()
#             sc=scaler.fit(test_data)
#             test_data=sc.transform(test_data)
            
            
            
    
    
#             dfasdf
#             print("temp befdore prediction \n",temp)
#             if u == 'updrs_4':
#                 temp["rating"]=0
                
#             if test_df[FEATURES].sum(axis=1) != 0:
#                 temp['rating']=0
            print("predictioing params ","u=",u,"m=",m)
    
            xgbmodel3 = pickle.load(open(model_dict1[u],'rb'))
#             xgbmodel3 = model_dict1[u]
        
            r3=xgbmodel3.predict(test_data)
            
#             print("average r3 ",r3)
#             temp.loc[mask,"rating"]=r3
            print("u here is ",u,"\n vm here is ",vm)
     
    
            if (u,vm) in model_dict.keys(): 
#                 print("works fine")
#                 asdf
                xgbmodel2 = pickle.load(open(model_dict[(u,vm)], 'rb'))
#                 xgbmodel2 = model_dict[(u,m)]
                r2=xgbmodel2.predict(test_data)
                print("prediction if vm ",vm,"= \n",r2)  

            if (u,m) not in model_dict.keys():
                if (u,vm) not in model_dict.keys():
#                     temp.loc[mask,"rating"]=r3
                    temp["rating"]=r3
                    print("(u,m) not in dictionary, average rating for ",u,m,"=",r3)
                else: 
#                     temp.loc[mask,"rating"]=r2
                    temp['rating']=r2
                    print("(u,m) not in dictionary, vm rating for ",u,vm,"=",r2)
        
            if (u,m) in model_dict.keys():
                print("month based model used for prediction \n",u,m)
                xgbmodel1 = pickle.load(open(model_dict[(u,m)], 'rb'))
#                 xgbmodel1= model_dict[(u,m)]
                r1=xgbmodel1.predict(test_data) 
                
                if np.sum(r1) > np.sum(r3):
                    temp['rating']=r1
                    print("r1 value greater than r3, r1 chosen \n",r1)
                else:
                
                    if (0.10*np.sum(r3) >=np.sum(r1)):
                        if (0.15*np.sum(r3) >= np.sum(r2)):                        
                            print("values when data not available ",vm,pm,u,m,r3,r1,0.15*np.sum(r3),np.sum(r1))
    #                         temp.loc[mask,"rating"] =  r3
                            temp['rating']=r3
                            print("r3 chosen instead of r2 because 0.15*r3 > r1 \n",r3)
                        else: 
    #                         temp.loc[/mask,"rating"] =  r2
                            print("r2 chosen instead of r3")
                            temp["rating"] =  r2

                    else: 
                    
                        if (np.sum(r2) > np.sum(r3)):
                            temp["rating"]=r2
                            print("r2 greater, hence r2 chosen \n",r2)
                        else:
                            
                            print("at combin r1 \n",r1,"\n r3",r3)
                            print("temp rating",temp.shape,len(r1),len(r3))
        #                     temp.loc[mask,"rating"] = r1*0.6 + r3*0.4
        #                     temp["rating"] = r1*0.6 + r3*0.4
                            temp["rating"] = r1*0.4 + r3*0.6
                            print("prediction based on combination of r1,r3 \n",temp['rating']) #temp.loc[mask,"rating"])

            print("The three results for ",vm,pm,m,u,"are \n",r1,"\n",r2,"\n",r3)
                
#             else:
                    
#                 temp.loc[mask,"rating"]=0
#                 print("rating for ",vm,pm,u,m,"is zero")
#             b[b[0].isin(a[0])]
#             temp[temp['visit_id'].isin(test['visit_id'])]
            if u=='updrs_4':
                    temp['rating']=0
    
            temp = temp[['prediction_id', 'rating']]   
            
            result = result.append(temp)  
            
            print("results for :\n","label=",u, "visitng month ",vm,"prediction period =",pm, 
                  "dict prediction month =",m,"\n",result,"\n")  
            
             
#             print("results in loop \n",result)

#                 test_df.loc[mask, 'result_' + str(u)] = model_dict[u].predict(new_test_df)
#     print("test_df with predictions \n",test_df.head())
    # Format for final submission
    
#     print("test df after predictions before df \n",test_df)
#     result = pd.DataFrame()

#     for m in [0, 6, 12, 24]:
#         for u in [1, 2, 3, 4]:

#             temp = test_df[["visit_id", "result_updrs_" + str(u)]].copy()  #visit_id
#             temp["prediction_id"] = temp["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
#             temp["rating"] = temp["result_updrs_" + str(u)]
#             temp = temp [['prediction_id', 'rating']]

#             result = result.append(temp)            
#     print("results befdore dup \n",result)
    result = result.drop_duplicates(subset=['prediction_id', 'rating'])
    
#     b[b[0].isin(a[0])]
    
#     ?result[result['visit_id'].isin(a[0])]?
    
    print("results GET PRED \n",result)

    return  result

In [21]:
import os
import pandas as pd
os.chdir(r"/kaggle/working")
# os.remove(r"/kaggle/working/submission.csv")

import amp_pd_peptide_310
amp_pd_peptide_310.make_env.func_dict['__called__'] = False
env = amp_pd_peptide_310.make_env()   # initialize the environment
iter_test = env.iter_test()  

master_testdf=pd.DataFrame()
 
# amp_pd_peptide_310.make_env.func_dict['__called__'] = False
# env = amp_pd_peptide_310.make_env()   # initialize the environment
# iter_test = env.iter_test()  
# Reference: https://www.kaggle.com/code/xarispanagiotopoulos/linearregression-xgb/notebook
for (test, test_peptides, test_proteins, sample_submission) in iter_test:
    
    print("test file \n",test)
    print("test_peptides \n",test_peptides)
    print("test prote \n",test_proteins)
    print("sample submissio \n",sample_submission)


    pro_pep_df = prepare_dataset(test_proteins,test_peptides)

    test_proteins = test_proteins.pivot(index=['visit_id', 'visit_month', 'patient_id'], 
              columns='UniProt', values='NPX').reset_index().rename_axis(None, axis=1).drop(columns=['visit_month', 'patient_id'])
    merged_df = test.merge(test_proteins, how='left', on='visit_id')

    test_df = test.merge(pro_pep_df, how='left', on='visit_id')
    
    
    
    # Add all the missing columns from FEATURES list and set the values as 0
    for col in FEATURES:
        if col not in test_df.columns:
            test_df[col] = 0
    test_df = test_df[['visit_id', 'patient_id'] + FEATURES] 
    master_testdf=pd.concat([master_testdf,test_df])
    
#     result = get_predictions(test_df, master_testdf, model_dict)
    result = get_predictions(test_df, master_testdf, model_dict)
    
    print("final results \n",result)
    os.chdir(r"/kaggle/working")
#     os.chdir(r'/kaggle/input/draftmod')

    env.predict(result)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
test file 
   visit_id  visit_month  patient_id updrs_test           row_id
0   3342_0            0        3342    updrs_1   3342_0_updrs_1
1   3342_0            0        3342    updrs_2   3342_0_updrs_2
2   3342_0            0        3342    updrs_3   3342_0_updrs_3
3   3342_0            0        3342    updrs_4   3342_0_updrs_4
4  50423_0            0       50423    updrs_1  50423_0_updrs_1
5  50423_0            0       50423    updrs_2  50423_0_updrs_2
6  50423_0            0       50423    updrs_3  50423_0_updrs_3
7  50423_0            0       50423    updrs_4  50423_0_updrs_4
test_peptides 
      visit_id  visit_month  patient_id UniProt  \
0     50423_0            0       50423  O00391   
1     50423_0            0       50423  O00391   
2     50423_0            0       50423  O00533   
3     50423_0            0       50423  O00533   
4     50423_0        

/tmp/ipykernel_20/1218289333.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_testdf=test_df[FEATURES].append(rep_df[FEATURES])
/tmp/ipykernel_20/1218289333.py:76: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df=test_df.append(rep_df)


new tdf 
     O00391    O00533   O00584  O14498   O14773   O14791    O15240   O15394  \
0      NaN       NaN      NaN       0      NaN      NaN       NaN      NaN   
1      NaN       NaN      NaN       0      NaN      NaN       NaN      NaN   
2      NaN       NaN      NaN       0      NaN      NaN       NaN      NaN   
3      NaN       NaN      NaN       0      NaN      NaN       NaN      NaN   
4  33127.9  490742.0  43615.3       0  16486.6  2882.42  124344.0  57688.1   
5  33127.9  490742.0  43615.3       0  16486.6  2882.42  124344.0  57688.1   
6  33127.9  490742.0  43615.3       0  16486.6  2882.42  124344.0  57688.1   
7  33127.9  490742.0  43615.3       0  16486.6  2882.42  124344.0  57688.1   
0      NaN       NaN      NaN       0      NaN      NaN       NaN      NaN   
1      NaN       NaN      NaN       0      NaN      NaN       NaN      NaN   
2      NaN       NaN      NaN       0      NaN      NaN       NaN      NaN   
3      NaN       NaN      NaN       0      NaN      Na

/tmp/ipykernel_20/1218289333.py:280: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(temp)


after mean           O00391         O00533        O00584  O14498        O14773  \
0  11641.264435  511164.883903  26505.529157       0  17688.295406   
1  11641.264435  511164.883903  26505.529157       0  17688.295406   
2  11641.264435  511164.883903  26505.529157       0  17688.295406   
3  11641.264435  511164.883903  26505.529157       0  17688.295406   
4  33127.900000  490742.000000  43615.300000       0  16486.600000   
5  33127.900000  490742.000000  43615.300000       0  16486.600000   
6  33127.900000  490742.000000  43615.300000       0  16486.600000   
7  33127.900000  490742.000000  43615.300000       0  16486.600000   
0  11641.264435  511164.883903  26505.529157       0  17688.295406   
1  11641.264435  511164.883903  26505.529157       0  17688.295406   
2  11641.264435  511164.883903  26505.529157       0  17688.295406   
3  11641.264435  511164.883903  26505.529157       0  17688.295406   

        O14791         O15240        O15394         O43505         O60888  \


...

/tmp/ipykernel_20/1218289333.py:280: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(temp)


after mean           O00391         O00533        O00584        O14498        O14773  \
0  14564.000000  616804.000000  61418.100000  27830.100000  14959.200000   
1  14564.000000  616804.000000  61418.100000  27830.100000  14959.200000   
2  14564.000000  616804.000000  61418.100000  27830.100000  14959.200000   
3  14564.000000  616804.000000  61418.100000  27830.100000  14959.200000   
4  11641.264435  511164.883903  26505.529157  27305.934884  17688.295406   
5  11641.264435  511164.883903  26505.529157  27305.934884  17688.295406   
6  11641.264435  511164.883903  26505.529157  27305.934884  17688.295406   
7  11641.264435  511164.883903  26505.529157  27305.934884  17688.295406   

        O14791         O15240        O15394         O43505         O60888  \
0  2853.120000  223373.000000  58863.000000  207497.000000  175485.000000   
1  2853.120000  223373.000000  58863.000000  207497.000000  175485.000000   
2  2853.120000  223373.000000  58863.000000  207497.000000  175485.00000

/tmp/ipykernel_20/1218289333.py:280: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(temp)


after mean           O00391         O00533        O00584        O14498        O14773  \
0  14564.000000  616804.000000  61418.100000  27830.100000  14959.200000   
1  14564.000000  616804.000000  61418.100000  27830.100000  14959.200000   
2  14564.000000  616804.000000  61418.100000  27830.100000  14959.200000   
3  14564.000000  616804.000000  61418.100000  27830.100000  14959.200000   
4  11641.264435  511164.883903  26505.529157  27305.934884  17688.295406   
5  11641.264435  511164.883903  26505.529157  27305.934884  17688.295406   
6  11641.264435  511164.883903  26505.529157  27305.934884  17688.295406   
7  11641.264435  511164.883903  26505.529157  27305.934884  17688.295406   

        O14791         O15240        O15394         O43505         O60888  \
0  2853.120000  223373.000000  58863.000000  207497.000000  175485.000000   
1  2853.120000  223373.000000  58863.000000  207497.000000  175485.000000   
2  2853.120000  223373.000000  58863.000000  207497.000000  175485.00000

/tmp/ipykernel_20/1218289333.py:280: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(temp)
